# 5分でできる音声認識　JuliusをColabのGPUで動かす



音声認識技術の向上には目を見張るものがあります。ただ、それを応用していろいろやろうとした時、クラウドベースのサービスだと認識結果の文字列のみの出力で物足りない場合があります。そのような時、特に日本語の場合、オープンソースの[Julius](https://julius.osdn.jp/index.php)が頼りになります。そこでJuliusのディクテーションをColabで動かしてみました。後でGPUを使うので、ここでcolabのランタイムのタイプをGPUにしてください。

まずは、[ディクテーションキット](https://julius.osdn.jp/index.php?q=dictation-kit.html)を[ここ](https://github.com/julius-speech/dictation-kit)から取得します。この時、ちょっとつまずきがありました。「git-lfs をインストール・設定してから clone して下さい」と書いてあるのですが、その方法が書いてありません。検索した所、以下の手順でした。

In [1]:
!apt-get install git-lfs
!git lfs install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 4s (473 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.


途中でエラーが出ていますが、大丈夫です。

それではディクテーションキットを取得します。取得サイズが2.1Gと表示されれば成功です。

In [2]:
%cd /content/
!git clone https://github.com/julius-speech/dictation-kit.git
!du -sh /content/dictation-kit

/content
Cloning into 'dictation-kit'...
remote: Enumerating objects: 343, done.
remote: Total 343 (delta 0), reused 0 (delta 0), pack-reused 343
Receiving objects: 100% (343/343), 326.24 MiB | 38.19 MiB/s, done.
Resolving deltas: 100% (94/94), done.
Filtering content: 100% (28/28), 851.93 MiB | 62.09 MiB/s, done.
2.1G	/content/dictation-kit


次に[こちら](https://soratobu.net/voicesample/)から音声サンプルを取得します。音声ファイルはサンプリング周波数16k、モノラル、１６ビットpcm形式に変換します。([再生](http://soratobu.net/wp3/wp-content/uploads/2020/01/fukuikazue_documentary.mp3))

---



In [3]:
%cd /content
!wget -q http://soratobu.net/wp3/wp-content/uploads/2020/01/fukuikazue_documentary.mp3
!ffmpeg -v 0 -y -i fukuikazue_documentary.mp3 -ar 16000 -ac 1 -acodec pcm_s16le sample.wav

/content


ディクテーションキットに含まれるシェルスクリプトを利用して音声認識を実行します。まずは深層学習版。

In [4]:
%%time
%cd /content/dictation-kit/
!./run-linux-dnn.sh -input stdin -nolog -spsegment < /content/sample.wav

/content/dictation-kit
STAT: include config: main.jconf
STAT: include config: am-dnn.jconf
STAT: parsing option string: "-htkconf model/dnn/config.lmfb -cvn -cmnload model/dnn/norm -cmnstatic"
Stat: para: parsing HTK Config file: model/dnn/config.lmfb
Stat: para: SOURCERATE=625
Stat: para: TARGETRATE=100000.0
Stat: para: WINDOWSIZE=250000.0
Stat: para: USEHAMMING=T
Stat: para: PREEMCOEF=0.97
Stat: para: NUMCHANS=40
Stat: para: ENORMALISE=F

世界で次々に起こる災害や以上気性が、長期的な地球      
温暖化の傾向と一致していると、世界気象機関が、
異例の警笛を鳴らしている。統計開始以来、最近四         
年間が、平均祇園の最高記録、一から読みを独占して
いる。

reached end of input on stdin
CPU times: user 372 ms, sys: 47.5 ms, total: 419 ms
Wall time: 34 s


次は統計モデル版。

In [5]:
%%time
%cd /content/dictation-kit/
!./run-linux-gmm.sh -input stdin -nolog -spsegment < /content/sample.wav

/content/dictation-kit
STAT: include config: main.jconf
STAT: include config: am-gmm.jconf

世界で次々に起こる災害や異常気象が、長期的な地球      
温暖化の傾向と一致していると、世界気象機関が、         
入江の警笛を鳴らしている。統計開始以来。最近四         
年間が、平均気温の最高記録、一から四運用、特選し      
ている。

reached end of input on stdin
CPU times: user 105 ms, sys: 22.6 ms, total: 127 ms
Wall time: 7.56 s


どちらも惜しい感じです。

run-linux-dnn.shの中身を覗いて他の候補も出力するようにしてみましょう。

In [ ]:
!cat run-linux-dnn.sh

#! /bin/sh

./bin/linux/julius -C main.jconf -C am-dnn.jconf -demo -dnnconf julius.dnnconf $*


In [6]:
%%time
%cd /content/dictation-kit/
!./bin/linux/julius -C main.jconf -C am-dnn.jconf -dnnconf julius.dnnconf -input stdin -nolog -spsegment -quiet -confnet < /content/sample.wav

/content/dictation-kit
STAT: include config: main.jconf
STAT: include config: am-dnn.jconf
STAT: parsing option string: "-htkconf model/dnn/config.lmfb -cvn -cmnload model/dnn/norm -cmnstatic"
Stat: para: parsing HTK Config file: model/dnn/config.lmfb
Stat: para: SOURCERATE=625
Stat: para: TARGETRATE=100000.0
Stat: para: WINDOWSIZE=250000.0
Stat: para: USEHAMMING=T
Stat: para: PREEMCOEF=0.97
Stat: para: NUMCHANS=40
Stat: para: ENORMALISE=F

pass1_best:  世界 で 次々 に 起こる 災害 や 異常 気象 が 、 長期 的 な 地球 温暖 化 の 傾向 と 一致 し て いる と 、 世界 気象 機関 が 、 異例 の 警笛 を 鳴らし て いる 。
sentence1:  世界 で 次々 に 起こる 災害 や 以上 気性 が 、 長期 的 な 地球 温暖 化 の 傾向 と 一致 し て いる と 、 世界 気象 機関 が 、 異例 の 警笛 を 鳴らし て いる 。
---- begin confusion network ---
(:1.000)  
(-:0.989)  (、:0.006)(ウ:0.004)
(世界:0.951)  (せかい:0.022)(政界:0.014)(正解:0.010)(青海:0.001)(石灰:0.001)(切開:0.001)(せっかい:0.000)(節介:0.000)(制海:0.000)
(で:0.982)  (って:0.009)(て:0.008)(出:0.000)(デ:0.000)(Ｄ:0.000)(に:0.000)
(-:1.000)  (、:0.000)
(つぎつぎ:0.514)  (次々:0.484)(次つぎ:0.002)
(に:1.000)  
(起こる:1.000)  
(災

pass1_bestで「異常気象」や「平均気温」が正しく認識されているのに、最終的な認識結果の中では候補にもあがってません。いろいろパラメータを変えてやってみましたが、今のところ、最終候補に残るようにはできていません。

統計モデルはどうでしょうか。

In [ ]:
%%time
%cd /content/dictation-kit/
!./bin/linux/julius -C main.jconf -C am-gmm.jconf -input stdin -nolog -spsegment -quiet -confnet < /content/sample.wav

/content/dictation-kit
STAT: include config: main.jconf
STAT: include config: am-gmm.jconf

pass1_best:  世界 で 次々 に 起こる 災害 や 異常 気象 が 、
sentence1:  世界 で 次々 に 起こる 災害 や 異常 気象 が 、
---- begin confusion network ---
(:1.000)  (-:0.000)
(-:0.970)  (、:0.020)(千:0.007)(セ:0.004)
(世界:0.641)  (政界:0.125)(正解:0.095)(せかい:0.033)(青海:0.026)(旋回:0.016)(前回:0.011)(全開:0.009)(石灰:0.008)(回:0.006)(浅海:0.005)(切開:0.005)(界:0.004)(水害:0.003)(制海:0.003)(世代:0.002)(世帯:0.002)(せっかい:0.002)(節介:0.002)(世凱:0.001)(街:0.001)
(で:0.854)  (に:0.086)(て:0.040)(って:0.014)(-:0.004)(へ:0.002)
(-:0.980)  (、:0.020)
(次々:0.955)  (つぎつぎ:0.027)(次つぎ:0.018)
(に:1.000)  
(起こる:1.000)  
(災害:1.000)  
(や:1.000)  
(異常:1.000)  
(気象:1.000)  
(が:1.000)  
(、:1.000)  
---- end confusion network ---
pass1_best: 、 長期 的 な 時期 温暖 化 の 結婚 と 一致 し て いる と 、
sentence1: 、 長期 的 な 地球 温暖 化 の 傾向 と 一致 し て いる と 、
---- begin confusion network ---
(、:1.000)  
(-:1.000)  (ソー:0.000)(双:0.000)(ア:0.000)
(長期:0.981)  (早期:0.009)(想起:0.003)(そう:0.002)(超:0.002)(相互:0.001)(そっち:0.001)(消費:0.001)(臓器:0.0

「入江」に対して「異例」も候補に入っています。「特選」に対しては残念ながら「独占」が候補に挙がっていません。

ところで、Julius 4.6ではGPU対応が謳われています。執筆時点でディクテーションキットに含まれているJuliusは4.5です。せっかくなので、colabのGPUを有効にして処理速度向上を確かめてみましょう。

Juliusのエンジンを取得してビルドします。

In [ ]:
!apt-get install build-essential zlib1g-dev libsdl2-dev
!git clone https://github.com/julius-speech/julius.git
%cd /content/julius
!CC=/usr/local/cuda-10.1/bin/nvcc CFLAGS=-O3 ./configure
!make
!make install

ビルドしたバイナリを使って音声認識させます。

In [ ]:
%%time
%cd /content/dictation-kit/
!julius -C main.jconf -C am-dnn.jconf -dnnconf julius.dnnconf -input stdin -nolog -spsegment -demo < /content/sample.wav

/content/dictation-kit
STAT: include config: main.jconf
STAT: include config: am-dnn.jconf
STAT: parsing option string: "-htkconf model/dnn/config.lmfb -cvn -cmnload model/dnn/norm -cmnstatic"
Stat: para: parsing HTK Config file: model/dnn/config.lmfb
Stat: para: SOURCERATE=625
Stat: para: TARGETRATE=100000.0
Stat: para: WINDOWSIZE=250000.0
Stat: para: USEHAMMING=T
Stat: para: PREEMCOEF=0.97
Stat: para: NUMCHANS=40
Stat: para: ENORMALISE=F

世界で次々に起こる災害や以上気性が、長期的な地球      
温暖化の傾向と一致していると、世界気象機関が、
異例の警笛を鳴らしている。統計開始以来、最近四         
年間が、平均祇園の最高記録、一から読みを独占して
いる。

1 files processed
CPU times: user 150 ms, sys: 24 ms, total: 174 ms
Wall time: 11.9 s


さすがGPU、精度は変わりませんが処理速度は3倍になっています。

ちなみにGoogleの音声認識はどうでしょうか。

In [ ]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 1.3 MB/s 


In [ ]:
%%time

import speech_recognition as sr

AUDIO_FILE = "/content/sample.wav"

# use the audio file as the audio source
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)  # read the entire audio file

result=r.recognize_google(audio, language='ja-JP')
print(result)

世界で次々に起こる災害や異常気象が長期的な地球温暖化の傾向と一致していると 世界気象機関が異例の警笛を鳴らしている統計開始以来最近 四年間が平均気温の最高記録 1位から4位を独占している
CPU times: user 13 ms, sys: 8.98 ms, total: 22 ms
Wall time: 1.47 s


認識結果が完璧な上に処理速度も速い、、、さすが、、、